In [1]:
from pytube import YouTube, exceptions
import json
import os
import pandas as pd

In [2]:
def resolution_generator(resolutions):
    for res in resolutions:
        yield res

In [3]:
def load_json(loc):
    with open(loc, 'r') as f:
        json_data = json.load(f)
        f.close()
    return json_data

In [4]:
data = load_json('data.json')

In [5]:
main_dir = r"C:\Users\Mohammed Faizan\OneDrive\Desktop\Faizan\Projects\iNueronChallenge28Aug\videos"

In [6]:
skipped_videos = []
for i, videoId in enumerate(data['videoId']):
    filename = f"{videoId}.mp4"
    if not os.path.isfile(os.path.join(main_dir, filename)):
        url = f"https://www.youtube.com/watch?v={videoId}"
        rgen = resolution_generator(['360p', '480p', '720p', '144p'])
        res = next(rgen)
        print(f"{i+1}: connecting to ", videoId)
        yt_video = YouTube(url)
        while True:
            try:
                print(f'{i+1}: Downloading with {res}')
                stream = yt_video.streams.filter(progressive=True, file_extension='mp4', res = res).first()
                stream.download(main_dir, filename = filename)
                print(f'{i+1}: Downloaded {videoId} in {stream.resolution}\n')
                break
            except StopIteration:
                print('Could not manually select the proper resolution\nRetrying with the highest resolution')
                try:
                    stream = yt_video.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
                    stream.download(main_dir, filename = filename)
                    print(f'{i+1}: Downloaded {videoId} in {stream.resolution}\n')
                except:
                    print('Retry with highest resolution failed')
                    skipped_videos.append(videoId)
                    break
            except exceptions.LiveStreamError as e:
                print(e)
                print('moving on...')
                break
            except Exception as e:
                print('--- ERROR ---')
                print(e)
                print(f'{i+1}: Error in ', videoId)
                res = next(rgen)
                continue
    else:
        print(f"{i+1}: {filename} already exists.")

1: 82fPl5l0vXY.mp4 already exists.
